In [1]:
import cv2
import numpy as np
from skimage.util import random_noise
from tqdm.notebook import tqdm
import os

In [2]:
# эффект бумаги

BG_COLOR = 209
BG_SIGMA = 5
CHANNELS = 3


def blank_image(width=1024, height=1024, background=BG_COLOR):
    img = np.full((height, width, CHANNELS), background, np.uint8)
    return img


def add_noise(img, sigma=BG_SIGMA):

    width, height, ch = img.shape
    n = noise(width, height, sigma=sigma)
    img = img + n
    return img.clip(0, 255)


def noise(width, height, ratio=1, sigma=BG_SIGMA):

    mean = 0
    h = int(height / ratio)
    w = int(width / ratio)

    result = np.random.normal(mean, sigma, (w, h, CHANNELS))
    if ratio > 1:
        result = cv2.resize(result, dsize=(width, height), interpolation=cv2.INTER_LINEAR)
    return result.reshape((width, height, CHANNELS))


def texture(image, sigma=BG_SIGMA, turbulence=2):
    result = image.astype(float)
    cols, rows, ch = image.shape
    ratio = cols
    while not ratio == 1:
        result += noise(cols, rows, ratio, sigma=sigma)
        ratio = (ratio // turbulence) or 1
    cut = np.clip(result, 0, 255)
    return cut.astype(np.uint8)

def get_paper(colour = 230):
    return add_noise(texture(blank_image(4096, 4096, colour)))

In [3]:
def put_image_on_paper(img, bg):
    mask = np.zeros_like(img)
    mask[:, :, 0] = mask[:, :, 1] = mask[:, :, 2] = (img[:, :, 0] == 255) * (img[:, :, 1] == 255) * (img[:, :, 2] == 255)
    mask = mask.astype(bool)
    return bg[:img.shape[0], :img.shape[1]] * mask + img * ~mask

def put_image_on_background(img, bg):
    mask = np.zeros_like(img)
    mask[:, :, 0] = mask[:, :, 1] = mask[:, :, 2] = (img[:, :, 0] == 1) * (img[:, :, 1] == 1) * (img[:, :, 2] == 1)
    mask = mask.astype(bool)
    return bg * mask + img * ~mask

In [4]:
# дефект печати
def print_defect(img):
    shifted_rows = np.random.binomial(1, 0.01, img.shape[0])
    for i in range(img.shape[0]):
        if shifted_rows[i] == 1:
            shift = np.random.randint(-25, 25)
            new_row = np.zeros_like(img[i])
            if shift > 0:
                new_row[shift:] = img[i][:-shift]
            elif shift < 0:
                new_row[:shift] = img[i][-shift:]
            else:
                new_row = img[i]
            img[i] = new_row
    return img

In [5]:
# стандартные преобразования
def channel_shift(img, value = 60):
    value = int(np.random.uniform(-value, value))
    img = img + value
    img[:,:,:][img[:,:,:]>255]  = 255
    img[:,:,:][img[:,:,:]<0]  = 0
    img = img.astype(np.uint8)
    return img

def rotation(img, angle = 45):
    angle = int(np.random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h), borderValue = (1, 1, 1))
    return img

def translation(img, shift = 500):
    w_shift = int(np.random.uniform(-shift, shift))
    h_shift = int(np.random.uniform(-shift, shift))
    h, w = img.shape[:2]
    M = np.array([[1, 0, w_shift], [0, 1, h_shift]], dtype=np.float32)

    img = cv2.warpAffine(img, M, (w, h), borderValue = (1, 1, 1))
    return img

def blur(img, shape = 10):
    kern = np.random.randint(1, shape)
    return cv2.blur(img, (kern, kern))

def resize(img, scale_percent = 100):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return img

In [8]:
def make_augmentations(img):
    bg = resize(cv2.imread('Backgrounds/bg' + str(np.random.randint(1, 5)) + '.png'), 200)
    new_img = resize(img, np.random.randint(30, 65))
    new_img = print_defect(put_image_on_paper(new_img, paper))
    trick = np.ones_like(bg)
    trick[500:500 + new_img.shape[0], 500:500 + new_img.shape[1]] = new_img
    new_img = translation(rotation(trick))
    return blur(put_image_on_background(new_img, bg))

In [ ]:
paper = get_paper()

In [9]:
for file in tqdm(os.listdir('Data/')):
    img = cv2.imread('Data/' + file)
    for i in range(10):
        cv2.imwrite('Result/' + file[:-4]+ '_' + str(i) + '.png', make_augmentations(img))